In [37]:
import pandas as pd
import math
import numpy as np

data = pd.read_csv("PlayTennis.csv")
features = [feat for feat in data]


features.remove("Target")


print("fea:",features)

class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        
        if (row["Target"] == "no"):
            neg += 1
        else:
            pos += 1
    print("pn",pos,neg)
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

def info_gain(examples, attr):
    print(attr)
    uniq = np.unique(examples[attr])
    print(uniq)
   
    gain = entropy(examples)
    print("gain(s): ",gain)

    for u in uniq:
        subdata = examples[examples[attr] == u]
     
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
    
    return gain

def ID3(examples, attrs):
    root = Node()
    max_gain = 0
    max_feat = ""
    for feature in attrs:
        print('Features: ',feature)
        gain = info_gain(examples, feature)
        print("gain: ",gain)
        print("maxgain: ",max_gain)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
        print("max: ",max_feat)
    root.value = max_feat
    uniq = np.unique(examples[max_feat])
    for u in uniq:
        subdata = examples[examples[max_feat] == u]
        
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Target"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root

def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

root = ID3(data, features)
printTree(root)

fea: ['Outlook', 'Temperature', 'Humidity', 'Wind']
Features:  Outlook
Outlook
['overcast' 'rain' 'sunny']
pn 11.0 3.0
gain(s):  0.74959525725948
pn 4.0 0.0
pn 4.0 1.0
pn 3.0 2.0
gain:  0.14499572535161176
maxgain:  0
max:  Outlook
Features:  Temperature
Temperature
['cool' 'hot' 'mild']
pn 11.0 3.0
gain(s):  0.74959525725948
pn 4.0 0.0
pn 2.0 2.0
pn 5.0 1.0
gain:  0.18529993369589964
maxgain:  0.14499572535161176
max:  Temperature
Features:  Humidity
Humidity
['high' 'normal']
pn 11.0 3.0
gain(s):  0.74959525725948
pn 4.0 3.0
pn 7.0 0.0
gain:  0.25698118924235414
maxgain:  0.18529993369589964
max:  Humidity
Features:  Wind
Wind
['strong' 'weak']
pn 11.0 3.0
gain(s):  0.74959525725948
pn 4.0 2.0
pn 7.0 1.0
gain:  0.04543164655064369
maxgain:  0.25698118924235414
max:  Humidity
pn 4.0 3.0
Features:  Outlook
Outlook
['overcast' 'rain' 'sunny']
pn 4.0 3.0
gain(s):  0.9852281360342516
pn 2.0 0.0
pn 1.0 1.0
pn 1.0 2.0
gain:  0.3059584928680419
maxgain:  0
max:  Outlook
Features:  Temperatur

In [41]:
def test_train_split(data):
    x = data.iloc[:,0:-2]
    y = data.iloc[:,-1]
    return x,y
    
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x,y = test_train_split(data)
x1 = le.fit_transform(data.iloc[:,0])
x2 = le.fit_transform(data.iloc[:,1])
x3 = le.fit_transform(data.iloc[:,2])
x4 = le.fit_transform(data.iloc[:,3])
y = le.fit_transform(data.iloc[:,4])

x = list(zip(x1,x2,x3,x4))

clf = DecisionTreeClassifier().fit(x,y)

df = pd.read_csv('PlayTennisTestData.csv')
df1 = np.array(df)
d = {0:'no',1:'no',2:'yes'}
x1test = le.fit_transform(df.iloc[:,0])
x2test = le.fit_transform(df.iloc[:,1])
x3test = le.fit_transform(df.iloc[:,2])
x4test = le.fit_transform(df.iloc[:,3])
xtest = list(zip(x1test,x2test,x3test,x4test))

y_pred = clf.predict(xtest)
y_pred_labels = [d[i] for i in y_pred]
for i in range(len(df1)):
    print("The test instance is: ",df1[i])
    print("The predicted label is: ",y_pred_labels[i])

0       no
1       no
2     yes 
3     yes 
4     yes 
5      no 
6     yes 
7      no 
8     yes 
9     yes 
10    yes 
11    yes 
12    yes 
13      no
Name: Target, dtype: object
[0 0 2 2 2 1 2 1 2 2 2 2 2 0]
The test instance is:  ['sunny' 'cool' 'normal' 'weak ']
The predicted label is:  yes
The test instance is:  ['sunny' 'mild' 'normal' 'strong']
The predicted label is:  yes
The test instance is:  ['overcast' 'mild' 'high' 'strong ']
The predicted label is:  yes
The test instance is:  ['overcast' 'hot' 'normal' 'weak ']
The predicted label is:  yes
The test instance is:  ['rain' 'mild' 'high' 'strong']
The predicted label is:  no
The test instance is:  ['rain' 'mild' 'normal' 'weak']
The predicted label is:  yes
The test instance is:  ['sunny' 'mild' 'high' 'weak']
The predicted label is:  no
